# Relativistic one body simulations

Version history

In [ ]:
# @title
# 2025-04-21 1-body v13.jpynb
# Amended notebook to show all configurations in one notebook

# 2025-01-09 1-body v12.jpynb
# Converted code to Jupyter Notebook

# 2024-08-05 M-units v12
# Introduced numeric calculation of metric tensor and Christoffel symbols. Significant speed up. Symbolic calculation remains an option
# Added two configurations "Photon deflected by spin"

# 2024-08-04 M-units v11
# Replace own Runge Kutta with solve_ivp

# 2024-07-12 M-units v10
# Re-integrated earlier examples of Mercury and S2 orbits that work on ISO units.
# Improved calculation of E, Lz and Q to work with ISO units

# 2024-07-10 M-units v9
# Introduced null geodesics. Calibrated light ray passing by the surface of the sun. Resulting deflection is 1.758 arcsec,
# consistent with expectations. Introduced solver for vt0 to make sure normalization condition is met exactly

# 2024-07-08 M-units v8
# Further refinement of initial values to better match ds=1

# 2024-07-08 M-units v7
# Correctly calibrated velocity-based time dilatation and conversion from local to global velocities. Trick was to
# introduce calculation of ds and recognize that this was not 1.
# This now closely matches the orbits from yukterez.net. E, Lz and Q also match closely

# 2024-07-06 M-units v6
# Added interactive 3D plot

# 2024-07-05 M-units v5
# Consistent use of Boyer-Lindquist coordinates established

# 2024-07-03 M-units v4
# Extended code to match the two example orbits (prograde and retrograde) on the Wikipedia site on Kerr metric
# For that purpose, included calculation of ZAMO (Zero Angular Momentum Observer) angular velocity

# 2024-07-01 M-units v3
# Use of EinsteinPy to symbolically determine and subsequently evaluate Christoffel symbols
# First time use of Kerr metric to demonstrate frame dragging effect (rotating black hole)
# Simulation considerably slower than in pervious versions (M-units v2 and earlier)

# 2024-06-26 M-units v2
# Introduced theta parameter in simulation to extend to 3 dimension (incl. 3D charts). Generally works well, with following issue observed:
# - Determination of perihel shift doesnt work anymore. Needs a new approach.
# - Coordinate singularity at the poles

# 2024-06-26 M-units v1
# Converted to M units. Introduced 3D chart

# 2024-06-24 S2-v4
# Extended simulation to include relativistic time, using Christoffel symbols (as for r and phi). This now leads the correct results!
# Perihelion shift of 713 arc seconds (ca. 12 arc minutes) per orbit
# SOLUTION NOW SUSTAINABLE

# 2024-06-24 S2-v3
# Attempt another fix. SOLUTION STILL UNSUSTAINABLE

# 2024-06-23 S2-v2
# Back to geodesic equation. Attempt to fix the too low (factor 3) perihelion shift by coordinate transformation.
# But needs unexplained factor of 2 (see code). SOLUTION UNSUSTAINABLE

# 2024-06-22 S2-v1
# Simulation of Star S2 orbit around Sagittarius A*. Commonly used mass for Sagittarius A* tweaked slightly to produce the 16 earth year orbit.
# Periastron shift closely matches the commonly referred 12 arcmins per orbit!
# Based on post newtonian approximation

# 2024-06-20 Mercury6
# Relativistic calculation based on sperical coordinates and Christoffel symbols. Converts to newton when c->inf
# Results in perihelion shift of 0.0346" per orbit, a factor 3 too low

Imports

In [ ]:
# @title
import math
import sys
import numpy as np
import sympy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # This is necessary for 3D plotting
try:
    import einsteinpy
except ImportError:
    !pip install einsteinpy
from einsteinpy.symbolic import MetricTensor, ChristoffelSymbols
from einsteinpy.symbolic.predefined import Schwarzschild, Kerr
import plotly.graph_objects as go
from scipy.optimize import newton
from scipy.integrate import solve_ivp
import plotly.io as pio
pio.renderers.default = 'colab'

sympy.init_printing()  # enables the best printing available in an environment
G=vc=M=year=rs=vt0_newton=t0=theta0=phi0=vr0=adjustZAMO=ds0=spin=rtol=analyzePeriastron=analyzeDeflection=name=symbolic=count=func_metric=func_ch = None


Helper functions

In [ ]:
# @title
def initialize_globals():
    global G, vc, M, year, rs, vt0_newton, t0, theta0, phi0, vr0, adjustZAMO, ds0, spin, rtol, analyzePeriastron, analyzeDeflection, name, symbolic, count

    G = vc = M = 1  # Default: M-units
    year = 1  # earth year in seconds
    rs =  2 * G * M / (vc**2) # Schwarzschild radius
    vt0_newton = 1 # 1
    t0 = 0 # s
    theta0 = 90 / 90*np.pi/2  # default: 90 (eqator plane)
    phi0  = 0 / 180*np.pi     # default: 0
    vr0 = 0.0 # m/s
    adjustZAMO = False
    ds0 = 1.0 # timelike geodesic
    spin = 0
    rtol = 1e-9
    analyzePeriastron = False
    analyzeDeflection = False
    name = "Orbit"
    symbolic = False # set True for symbolic calculation of metric tensor and Christoffel symbols (takes much longer)
    count = 0
    return

def mt_num(r, theta, phi, c, r_s, a): # Calculate numeric Kerr Metric Tensor
    mt = np.zeros(16)
    mt = mt.reshape(4,4)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    fac1 = a**2*cos_theta**2 + r**2
    mt[0][0] = -r*r_s/(a**2*np.cos(theta)**2 + r**2) + 1
    mt[1][1] = (-a**2*cos_theta**2 - r**2)/(c**2*(a**2 + r**2 - r*r_s))
    mt[2][2] = (-a**2*cos_theta**2 - r**2)/c**2
    mt[3][3] = -(a**2*r*r_s*sin_theta**2/fac1 + a**2 + r**2)*sin_theta**2/c**2
    mt[0][3] = mt[3][0] = a*r*r_s*sin_theta**2/(c*fac1)
    return mt

def conservedQuantities_rel_fast(y, c, r_s, a): # Fast approach, avoiding tensors
    t, r, theta, phi, vt, vr, vtheta, vphi = y
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    fac1 = a**2*cos_theta**2 + r**2
    mt00 = -r*r_s/(a**2*np.cos(theta)**2 + r**2) + 1
    mt11 = (-a**2*cos_theta**2 - r**2)/(c**2*(a**2 + r**2 - r*r_s))
    mt22 = (-a**2*cos_theta**2 - r**2)/c**2
    mt33 = -(a**2*r*r_s*sin_theta**2/fac1 + a**2 + r**2)*sin_theta**2/c**2
    mt03 = a*r*r_s*sin_theta**2/(c*fac1)
    E = mt00*vt + mt03 * vphi # Energy from metric tensor
    Lz = -mt03*vt + -mt33 * vphi # Lz from metric tensor
    p_theta = mt22 * vtheta # impulse of theta coordinate
    Q = p_theta**2 + np.cos(theta)**2*(a**2*(1 - E**2) + Lz**2/np.sin(theta)**2) # Carter constant
    ds = mt00*vt**2 + mt11*vr**2 + mt22*vtheta**2 + mt33*vphi**2 + 2*mt03*vt*vphi # ds
    return [E, Lz, Q, ds]

def ch_num_fast(r, theta, phi, c, r_s, a, vt, vr, vtheta, vphi): # Fast approach, avoiding tensors
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    tan_theta = np.tan(theta)
    fac0 = a**2 + r**2
    fac1 = a**2*cos_theta**2 + r**2
    fac2 = a**2 + r**2 - r*r_s
    fac3 = a**2*np.cos(2*theta) + a**2 + 2*r**2
    fac4 = sin_theta**4
    # 0jk
    at  = (2*vt*vr) * (r_s*(-a**4*cos_theta**2 - a**2*r**2*cos_theta**2 + a**2*r**2 + r**4)/(2*fac1**2*fac2))
    at += (2*vt*vtheta) * (-2*a**2*r*r_s*np.sin(2*theta)/fac3**2)
    at += (2*vr*vphi) * (a*r_s*(a**4*cos_theta**2 - a**2*r**2*cos_theta**2 - a**2*r**2 - 3*r**4)*sin_theta**2/(2*c*fac1**2*fac2))
    at += (2*vtheta*vphi) * (a**3*r*r_s*sin_theta**3*cos_theta/(c*fac1**2))
    # 1jk
    ar  = vt**2 * (r_s*c**2*(-a**2*cos_theta**2 + r**2)*fac2/(2*fac1**3))
    ar += vr**2 * ((r*fac2 + (-2*r + r_s)*fac1/2)/(fac1*fac2))
    ar += vtheta**2 * (r*(-fac2)/fac1)
    ar += vphi**2 * ((-fac2)*(-2*a**2*r**2*r_s*sin_theta**2 + a**2*r_s*fac1*sin_theta**2 + 2*r*fac1**2)*sin_theta**2/(2*fac1**3))
    ar += (2*vt*vphi) * (a*r_s*c*(a**2*cos_theta**2 - r**2)*fac2*sin_theta**2/(2*fac1**3))
    ar += (2*vr*vtheta) * (-a**2*np.sin(2*theta)/fac3)
    # 2jk
    atheta  = vt**2 * (-4*a**2*r*r_s*c**2*np.sin(2*theta)/fac3**3)
    atheta += vr**2 * (a**2*np.sin(2*theta)/(fac2*fac3))
    atheta += vtheta**2 * (-a**2*np.sin(2*theta)/fac3)
    atheta += vphi**2 * ((-a**2*r*r_s*fac0*sin_theta**2 - fac1*(a**2*r*r_s*sin_theta**2 + fac0*fac1))*sin_theta*cos_theta/fac1**3)
    atheta += (2*vt*vphi) * (4*a*r*r_s*c*fac0*np.sin(2*theta)/fac3**3)
    atheta += (2*vr*vtheta) * (r/fac1)
    # 3jk
    aphi  = (2*vt*vr) * (a*r_s*c*(-a**2*np.cos(2*theta) - a**2 + 2*r**2)/(fac2*fac3**2))
    aphi += (2*vt*vtheta) * (-a*r*r_s*c/(fac1**2*tan_theta))
    aphi += (2*vr*vphi) * ((2*a**4*r*fac4 + 4*a**4*r*cos_theta**2 - 2*a**4*r - a**4*r_s*fac4 - a**4*r_s*cos_theta**2 + a**4*r_s + 4*a**2*r**3*cos_theta**2 - a**2*r**2*r_s*cos_theta**2 - a**2*r**2*r_s + 2*r**5 - 2*r**4*r_s)/(2*fac1**2*fac2))
    aphi += (2*vtheta*vphi) * ((a**4*fac4/tan_theta - a**4/tan_theta + 2*a**4*cos_theta**3/sin_theta + 2*a**2*r**2*cos_theta**3/sin_theta + a**2*r*r_s/tan_theta - a**2*r*r_s*cos_theta**3/sin_theta + r**4/tan_theta)/fac1**2)
    return [-at, -ar, -atheta, -aphi]

# solves for correct vt0 (coordinate time velocity). _ds0 is the target value (1.0 for timelike, 0.0 for nul geodesics)
def vt0func(_r, _phi, _theta, _vr, _vphi, _vtheta, _ds0):
    if symbolic: mt = func_metric(_r, _theta, _phi, vc) # evaluate the metric tensor
    else: mt = mt_num(_r, _theta, _phi, vc, rs, spin)
    def ds_func(x): # x represents the unknow variable vt
        ret = 0 # initiate the result
        u = np.array([x, _vr, _vtheta, _vphi]) # define coordinate velocities with x for vt
        for nu in range(4):
            for sigma in range(4):
                ret += mt[nu][sigma] * u[nu] * u[sigma]
        ret -= _ds0 # deduct target value
        return float(ret)

    _vt0 = np.sqrt(-ds_func(0) / mt[0][0]) # determine approximation
    _vt0 = newton(ds_func, x0=_vt0) # find the vt that leads to ds=ds_target
    return _vt0

def acc_rel(t, y):
    global count
    _t, _r, _theta, _phi, _vt, _vr, _vtheta, _vphi = y
    if symbolic: # uses symbolic tensors
        ch_tensor = func_ch(_r, _theta, _phi, vc) # evaluate the Christoffel symbols
        u = np.array([_vt, _vr, _vtheta, _vphi]) # define coordinate velocities
        acc = sympy.zeros(4, 1) #define empty acc vector
        for mu in range(4):  # contract Christoffels with velocities
            for nu in range(4):
                for sigma in range(4):
                    acc[mu] -= ch_tensor[mu][nu][sigma] * u[nu] * u[sigma]
        at = float(acc[0])
        ar = float(acc[1])
        atheta = float(acc[2])
        aphi = float(acc[3])
    else: # uses numeric tensor
        at, ar, atheta, aphi = ch_num_fast(_r, _theta, _phi, vc, rs, spin, _vt, _vr, _vtheta, _vphi) # evaluate the Christoffel symbols
    count += 1
    return [_vt, _vr, _vtheta, _vphi, at, ar, atheta, aphi]

def acc_newton(t, y):
    _t, _r, _theta, _phi, _vt, _vr, _vtheta, _vphi = y
    at = 0
    ar = -G * M / (_r**2) + _r * np.sin(_theta)**2 * (_vphi**2) + _r * (_vtheta**2)
    aphi = -2 / _r * (_vr*_vphi) - 2*np.cos(_theta)/np.sin(_theta) * (_vtheta*_vphi)
    atheta = -2/_r * (_vr*_vtheta) + np.sin(_theta)*np.cos(_theta) * (_vphi**2)
    return [_vt, _vr, _vtheta, _vphi, at, ar, atheta, aphi]

def distance_rel(y): # distance in m
    return np.linalg.norm(position_rel(y))

def position_rel(y):
    xx = np.sqrt(y[1]**2 + spin**2) * np.sin(y[2])*np.cos(y[3])
    xy = np.sqrt(y[1]**2 + spin**2) * np.sin(y[2])*np.sin(y[3])
    xz = y[1] * np.cos(y[2])
    return np.array([xx,xy,xz])

def angle_rel(y_prior, y): # determines the angle (in deg) between 2 states, as defined by their x vector
    x_prior = position_rel(y_prior)
    x = position_rel(y)
    cosPhi = np.dot(x_prior, x)/np.linalg.norm(x)/np.linalg.norm(x_prior)
    return np.arccos(cosPhi)/np.pi*180 # deg

def position_newton(y):
    xx = y[1] * np.sin(y[2])*np.cos(y[3])
    xy = y[1] * np.sin(y[2])*np.sin(y[3])
    xz = y[1] * np.cos(y[2])
    return np.array([xx,xy,xz])

def angle_newton(y_prior, y): # determines the angle (in deg) between 2 states, as defined by their x vector
    x_prior = position_newton(y_prior)
    x = position_newton(y)
    cosPhi = np.dot(x_prior, x)/np.linalg.norm(x)/np.linalg.norm(x_prior)
    return np.arccos(cosPhi)/np.pi*180 # deg

def dr_func(t, y): # for periastron detection
    _t, _r, _theta, _phi, _vt, _vr, _vtheta, _vphi = y
    return _vr
dr_func.terminal = False  # Do not terminate integration at the event
dr_func.direction = +1  # Trigger on positive second derivative of radial velocity (approaching periastron)

Main function to simulate orbit

In [ ]:
# @title
def simulate_orbit():
    global count, name, ds0, factor, spin,r0, x0, vtheta0, vphi0, vr0, tau_max, analyzeDeflection, func_metric, func_ch
    print(name)

    # ZAMO
    if adjustZAMO:
        vlocal = np.sqrt((vtheta0*r0)**2 + (vphi0*x0)**2)
        lorenzV = 1/np.sqrt(1 - vlocal**2)
        sigma_zamo = r0**2 + spin**2 * np.cos(theta0)**2
        delta_zamo = r0**2 - rs*r0 + spin**2
        xi_zamo = (spin**2 + r0**2)**2 - spin**2 * np.sin(theta0)**2 * delta_zamo
        omega_zamo = rs*spin*r0/xi_zamo # ZAMO angular velocity in 1/s
        alpha = np.sqrt(xi_zamo/delta_zamo/sigma_zamo) # Gravitational red shift
        vphi0 += omega_zamo*alpha # adjust local velocity with omega
        vt0_rel = alpha*lorenzV   # 1
        vphi0 *= lorenzV
        vtheta0 *= lorenzV
        #print(f"vy0={vphi0*x0/vt0_rel/lorenzV}, vz0={-vtheta0*r0/vt0_rel/lorenzV}")

    if symbolic: # Define metric
        c, a, r_s, r, phi, theta, t = sympy.symbols('c a r_s r phi theta t') # Initialize symbolic variables
        metric = Kerr() # coordinates: [t, r, theta, phi] in this order
        #print(metric)
        #print()
        metric_tensor = metric.tensor()
        metric_tensor = metric_tensor.subs({r_s: rs, a: spin})
        func_metric = sympy.lambdify((r, theta, phi, c), metric_tensor, modules=['numpy']) # coordinate t is excluded as it doesnt feature in the expressions
        print("Calculate symbolic Christoffel symbols")
        ch = ChristoffelSymbols.from_metric(metric) # derive Christoffel symbols
        ch = ch.simplify() # simplify Christoffels
        #print(ch)
        #print()
        print(f"Substitute r_s={rs:.4e} and a={spin:.3f}")
        ch = ch.subs({r_s: rs, a: spin}) # substitute r_s and a with its value since these does not change
        func_ch = sympy.lambdify((r, theta, phi, c), ch, modules=['numpy']) # coordinate t is excluded as it doesnt feature in the expressions
    else: print("No symbolic calculation")

    # Time step, resolution and duration
    res = 10000 # number of data points
    t_span = (0, tau_max)  # period in s
    t_eval = np.linspace(t_span[0], t_span[1], res)
    dt = t_span[1] / res # s

    # Initial vectors for integration
    vt0_rel = vt0func(r0, phi0, theta0, vr0, vphi0, vtheta0, ds0)
    y0_rel = [t0, r0, theta0, phi0, vt0_rel, vr0, vtheta0, vphi0]
    y0_newton = [t0, x0, theta0, phi0, vt0_newton, vr0, vtheta0, vphi0]
    count = 0
    #cQ0 = conservedQuantities_rel(y0_rel) # [E,J,Q,ds]
    cQ0 = conservedQuantities_rel_fast(y0_rel, vc, rs, spin) # [E,J,Q,ds]
    normD0 = x0 # m
    normE0 = abs(cQ0[0]) # J
    normLz0 = max(abs(cQ0[1]), 1e-100) # kg m^2 s^-1
    normQ0 = abs(cQ0[2]) # kg m^2 s^-1
    print("Initial situation:")
    print(f"  r0={normD0:.3e}m, spin={spin}, vt_rel={vt0_rel:.4f}, E0={cQ0[0]:.3e}J, Lz0={cQ0[1]:.3e}kg*m^2/s, Q0={cQ0[2]:.3e}, ds0={cQ0[3]:.3e}")

    # Solving differential equation
    sol_rel = solve_ivp(acc_rel, t_span, y0_rel, t_eval=t_eval, events=dr_func, method='DOP853', rtol=rtol)
    p_rel_times = sol_rel.t_events[0]
    p_rel_states = sol_rel.y_events[0]
    n = len(sol_rel.y[0])-1
    print("Differenial Equation solution:")
    print(f"  Data points: {len(sol_rel.y[0])}, #acceleration calls={count}")
    print(f"Periastron points: {len(p_rel_states)}")

    per_n = len(p_rel_states)
    if analyzePeriastron: # periastron analysis
        timeFirstToLast = (p_rel_times[per_n-1] - p_rel_times[0])
        deltaPhi = 0 # initialize deltaPhi (deg)
        for k in range(per_n):
            #per_cQ = conservedQuantities_rel(p_rel_states[k])
            per_cQ = conservedQuantities_rel_fast(p_rel_states[k], vc, rs, spin)
            per_r = distance_rel(p_rel_states[k])
            if k==0: # memorize first periastron as a reference
                first_per_cQ = per_cQ
                first_per_r = per_r
            if k>0: # determine changes of orbital period
                deltaPhi += angle_rel(p_rel_states[k-1], p_rel_states[k])
                orbitPeriod = (p_rel_times[k]-p_rel_times[k-1])
                if k==1: firstOrbitPeriod = orbitPeriod # memorize first orbital period
                if deltaPhi > 1:    print(f"  {k}: deltaPhi={deltaPhi:.1f} deg")
                elif deltaPhi*60 > 1: print(f"  {k}: deltaPhi={deltaPhi*60:.1f} arcmin")
                else:                 print(f"  {k}: deltaPhi={deltaPhi*3600:.1f} arcsec")
        if k>1: # at least 2 periastron points detected
            deltaOrbitPeriod = orbitPeriod-firstOrbitPeriod
            deltaE = per_cQ[0] - first_per_cQ[0]
            deltaLz = per_cQ[1] - first_per_cQ[1]
            deltaQ = per_cQ[2] - first_per_cQ[2]
            deltaR = per_r - first_per_r
            print("Numerical results (between first and last periastron):")
            print(f"  deltaE={deltaE:.3e} J in {timeFirstToLast:.3e} s, dE/dt={deltaE/timeFirstToLast:.3e} W. deltaE/E0={deltaE/normE0:.3e} ")
            print(f"  deltaLz={deltaLz:.3e} kg m^2 s^-1, dLz/dt={deltaLz/timeFirstToLast:.3e} kg m^2 s^-2")
            print(f"  deltaQ={deltaQ:.3e}, dQ/dt={deltaQ/timeFirstToLast:.3e}")
            print(f"  Orbit decay dT={deltaOrbitPeriod:.3e} s in {timeFirstToLast:.3e} s, {deltaOrbitPeriod/timeFirstToLast*year:.3e} s/year")
            print(f"  Orbit decay deltaR={deltaR:.3e} m in {timeFirstToLast:.3e} s, dr={deltaR/timeFirstToLast*year:.3e} m/year. deltaR/r0={deltaR/r0:.3e}")

    if analyzeDeflection: # Deflection analysis
        if per_n == 1:  # Check if there is only one periastron
            m = int(n/10)
            d_start = position_rel(sol_rel.y[:,m]) - position_rel(sol_rel.y[:,0])
            d_end = position_rel(sol_rel.y[:,-1]) - position_rel(sol_rel.y[:,-1-m])
            d_start /= np.linalg.norm(d_start)
            d_end /= np.linalg.norm(d_end)
            deflection = np.arccos(np.dot(d_start, d_end))/np.pi*180*3600
            print(f"Deflection={deflection:.3f} arcsec, b={distance_rel(p_rel_states[0]):.3e}")

    # Calculate coordinates and conserved quantities
    r2 = np.array([position_rel(sol_rel.y[:,k]) for k in range(len(sol_rel.y[0]))]).T
    r1 = np.array([np.zeros(3) for k in range(len(sol_rel.y[0]))]).T
    #cQ = [conservedQuantities_rel(sol_rel.y[:, k]) for k in range(len(sol_rel.y[0]))]
    cQ = [conservedQuantities_rel_fast(sol_rel.y[:, k], vc, rs, spin) for k in range(len(sol_rel.y[0]))]
    print("Conserved Quantities determined")

    # HTML graph
    # Find the range and midpoint of data on all three axes
    max_range = np.array([max(r1[0]), max(r1[1]), max(r1[2]), max(r2[0]), max(r2[1]), max(r2[2])]).max()
    min_range = np.array([min(r1[0]), min(r1[1]), min(r1[2]), min(r2[0]), min(r2[1]), min(r2[2])]).min()
    chartL = (max_range-min_range)*0.2
    max_range += chartL
    min_range -= chartL

    def annotationsFunc(k):
        return [dict(
                    text=f"E={cQ[k][0]/normE0:.3f}, Lz={cQ[k][1]/normLz0:.3f}, Q={cQ[k][2]/normQ0:.3f}, ds={cQ[k][3]:.3f}",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.5, y=-0.02,
                    xanchor='center', yanchor='top', align='center',
                    font=dict(size=12, color="black"),
                ), dict(
                    text=f"d={normD0:.3e}m, spin={spin}, E0={cQ0[0]:.3e}J, Lz0={cQ0[1]:.3e}kg*m^2/s, Q0={cQ0[2]:.3e}, ds0={cQ0[3]:.3e}",
                    xref="paper", yref="paper",
                    x=0.15, y=1.05,  # Position the subtitle below the main title
                    xanchor='left', yanchor='top', align='center',
                    showarrow=False,
                    font=dict(size=12, color="black"),
                )
            ]

    fig = go.Figure(
        data=[ # Create a Plotly figure
            go.Scatter3d(x=[r1[0][0]], y=[r1[1][0]], z=[r1[2][0]], mode='markers', name = 'Body 1', marker=dict(size=10, color='red')),
            go.Scatter3d(x=[r2[0][0]], y=[r2[1][0]], z=[r2[2][0]], mode='markers', name = 'Body 2', marker=dict(size=10, color='blue')),
            #go.Scatter3d(x=[0, cQ[0][1][0]/normLz0*chartL], y=[0, cQ[0][1][1]/normLz0*chartL], z=[0, cQ[0][1][2]/normLz0*chartL], mode='lines', name='Angular Impulse', line=dict(color='black', width=4)),
            #go.Scatter3d(x=[r1[0][0], r1[0][0]+S1[0][0]/normS10*chartL], y=[r1[1][0], r1[1][0]+S1[1][0]/normS10*chartL], z=[r1[2][0], r1[2][0]+S1[2][0]/normS10*chartL], mode='lines', name='S1', line=dict(color='red', width=4)),
            #go.Scatter3d(x=[r2[0][0], r2[0][0]+S2[0][0]/normS20*chartL], y=[r2[1][0], r2[1][0]+S2[1][0]/normS20*chartL], z=[r2[2][0], r2[2][0]+S2[2][0]/normS20*chartL], mode='lines', name='S2', line=dict(color='blue', width=4)),
            go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', name='Origin', marker=dict(size=4, color='black')),
            go.Scatter3d(x=r1[0], y=r1[1], z=r1[2], mode='lines', name = 'Trj 1', line=dict(color='red', width=1)),
            go.Scatter3d(x=r2[0], y=r2[1], z=r2[2], mode='lines', name = 'Trj 2', line=dict(color='blue', width=1))
        ],
        layout=go.Layout(
            title = name,
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                xaxis=dict(range=[min_range, max_range]),
                yaxis=dict(range=[min_range, max_range]),
                zaxis=dict(range=[min_range, max_range]),
                aspectmode='cube',
                aspectratio=dict(x=1.8, y=1.8, z=1.8),
            ),
            annotations=annotationsFunc(0),
            updatemenus=[{
                "type": "buttons",
                "buttons": [
                    {
                        "label": "Play",
                        "method": "animate",
                        "args": [None, {"frame": {"duration": 0, "redraw": True}, "fromcurrent": True}]
                    },
                    {
                        "label": "XY Plane",
                        "method":"relayout",
                        "args": [{"scene.camera.eye": {"x": 0, "y": 0, "z": 1},
                            "scene.camera.up": {"x": 0, "y": 1, "z": 0},
                            "scene.camera.projection.type": "orthographic",
                            "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
                        }]
                    },
                    {
                        "label": "XZ Plane",
                        "method": "relayout",
                        "args": [{"scene.camera.eye": {"x": 0, "y": -1, "z": 0},
                            "scene.camera.up": {"x": 0, "y": 0, "z": -1},
                            "scene.camera.projection.type": "orthographic",
                            "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
                        }]
                    },
                    {
                        "label": "YZ Plane",
                        "method": "relayout",
                        "args": [{"scene.camera.eye": {"x": 1, "y": 0, "z": 0},
                            "scene.camera.projection.type": "orthographic",
                            "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
                        }]
                    },
                    {
                        "label": "3D View",
                        "method": "relayout",
                        "args": [{"scene.camera.eye": {"x": 1.5, "y": -1.5, "z": 0.5},
                            "scene.camera.up": {"x": 0, "y": 0, "z": -1},
                            "scene.camera.projection.type": "perspective",
                            "scene.aspectratio": {"x": 1.2, "y": 1.2, "z": 1.2}
                        }]
                    }
                ],
            }],
            sliders=[{
                "steps": [
                    {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                    "label": f"{i*dt:.3e}s",
                    "method": "animate"
                    } for i in range(0,n,10)
                ],
            }],
        ),
        frames=[go.Frame(
            data=[
                go.Scatter3d(x=[r1[0][k+1]], y=[r1[1][k+1]], z=[r1[2][k+1]], mode='markers', name = 'Body 1', marker=dict(size=10, color='red')),
                go.Scatter3d(x=[r2[0][k+1]], y=[r2[1][k+1]], z=[r2[2][k+1]], mode='markers', name = 'Body 2', marker=dict(size=10, color='blue')),
                #go.Scatter3d(x=[0, cQ[k+1][1][0]/normLz0*chartL], y=[0, cQ[k+1][1][1]/normLz0*chartL], z=[0, cQ[k+1][1][2]/normLz0*chartL], mode='lines', name='Angular Impulse', line=dict(color='black', width=4)),
                #go.Scatter3d(x=[r1[0][k+1], r1[0][k+1]+S1[0][k+1]/normS10*chartL], y=[r1[1][k+1], r1[1][k+1]+S1[1][k+1]/normS10*chartL], z=[r1[2][k+1], r1[2][k+1]+S1[2][k+1]/normS10*chartL], mode='lines', name='S1', line=dict(color='red', width=4)),
                #go.Scatter3d(x=[r2[0][k+1], r2[0][k+1]+S2[0][k+1]/normS10*chartL], y=[r2[1][k+1], r2[1][k+1]+S2[1][k+1]/normS10*chartL], z=[r2[2][k+1], r2[2][k+1]+S2[2][k+1]/normS10*chartL], mode='lines', name='S2', line=dict(color='blue', width=4))
            ],
            layout=go.Layout(annotations=annotationsFunc(k)),
            name=f"frame{k}"
        ) for k in range(0,n, 10)]

    )
    fig.update_layout(autosize=True)  # Disable autosizing to use specified size
    fig.show()
    fig.write_html('3D-orbit-animated.html') # Save to HTML
    return

__Mercury orbit around Sun__  
Mercury's orbit shows a perihelion shift of 574 arcsec per Julian century. 532 arcsec are explained by the tugs of other solar bodies (ie planets). This anomalous rate of precession (43 arcsec) of the perihelion of Mercury's orbit was first recognized in 1859 as a problem in celestial mechanics, by Urbain Le Verrier. A number of ad hoc and ultimately unsuccessful solutions (such as an additional planet "Vulcan" in the solar system) were proposed, but they tended to introduce more problems.
Einsteins theory of General Relativity precisely explains the 43 arcsec. Einsteins 1915 paper already included the quantification of this effect and was seen as an important confirmation of this new theory.
The simulation leads to precisely this effect (0.1 arcsec per Mercury orbit around the sun, times 400 orbits per Julian century).

In [ ]:
initialize_globals()
name = "Mercury orbit around Sun"
# results in perihelion shift of 0.1 arcsec per orbit (correct)
G = 6.67430e-11  # gravitational constant (m^3 kg^-1 s^-2)
vc = 2.99792e8  # speed of light (m/s)
M = 1.989e30  # mass of the Sun (kg)
spin = 0
year = 3.1536e7  # earth year in seconds
rs = 2 * G * M / (vc**2) # Schwarzschild radius in m
r0  = 4.601e10  # perihelion distance in m
x0 = np.sqrt(r0**2 + spin**2)
vphi0 = 58980.0 / r0  # perihelion velocity in m/s
vtheta0 = 0 # orbit in equatorial plane
vt0_rel = 1/np.sqrt(1 - rs/r0)   # 1
tau_max = 10 * year  # simulate for 10 year (ca 40 Mercury orbits)
analyzePeriastron = True # should be 0.1 arsec per orbit
simulate_orbit()

__Photon passing by the sun near its surface__  
General Relativity predicts that a light ray, passing by the surface of the sun hitting the earth, should be deflected by 1.7 arcsec. This number was first correctly determined bei Einstein in 1915. The first observation of light deflection was performed by noting the change in position of stars as they passed near the Sun on the celestial sphere. The observations were performed by Arthur Eddington and his collaborators (see Eddington experiment) during the total solar eclipse of May 29, 1919, when the stars near the Sun (at that time in the constellation Taurus) could be observed. The result was considered spectacular news and made the front page of most major newspapers. It made Einstein and his theory of general relativity world-famous.
The simulation leads to the correct deflection of 1.7 arcsec.

In [ ]:
# @title
initialize_globals()
name = "Photon passing by the sun near its surface"
# Impact parameter in M-Units: b = r_sun / rs_sun * 2 = 6.955e8 / 2.95e3 * 2 = 471520
# Results in correct deflection of 1.758 arcsec
ds0 = 0.0 # null geodesic
factor = 12000 # chosen such that photon passes by the sun
spin = 0 # Sun (no significant rotation)
r0 = 4715200  # 10 times the intended impact parameter of 471520
x0 = np.sqrt(r0**2 + spin**2)
vtheta0 = 0 #
vphi0 = 0.09/x0*factor # atan = 0.1
vr0 = -0.9*factor #atan = 0.1
tau_max = 1000 * year  # simulation period
analyzeDeflection = True # should be 1.758 arcsec
simulate_orbit()

__Star S2 orbit around Sagittarius A*__
Sagittarius A* (SgtA*) is the central BH in our galaxy, the milkyway. Its mass is 4.2 million sun masses, and its Schwarzschild radius is approx. 0.1 au. S2 is a star orbiting SgtA* in a highy eccentric orbit. Its mass is 15 sun masses, and the periastron distance is 17 light hours (120 au). The orbital period is 17 years. S2 reaches 2.56% the speed of light at its periastron. The simulation correctly captures the periastron shift of 713 arc seconds per orbit. The one body simulation does not account for GW radiation power nor orbital decay.  
Historical context: SgtA*, only 27'000 light years away from earth, has been identified as a supermassive BH in the 1980ies, by the work of Reinhard Genzel and Andrea Ghez who were awared the Nobel Price for their discoveries in 2020. SgtA* is not directly visible. It has been identified by orbiting objects such as its accretion disk and several stars (called the S group), amongst it S2. A picture of SgtA* accretion disk has been published in 2022.

https://cdn.eso.org/images/original/eso2208-eht-mwa.tif

In [ ]:
initialize_globals()
name = "Star S2 orbit around Sagittarius A*"
# results in periastron shift of 717 arcsec per orbit (correct)
G = 6.67430e-11  # gravitational constant (m^3 kg^-1 s^-2)
vc = 2.99792e8  # speed of light (m/s)
M = 4.211e6 * 1.989e30  # Ca 4 million times mass of the Sun (kg). Exact mass determined such that orbit takes 16 earth years
spin = 0
year = 3.1536e7  # earth year in seconds
rs = 2 * G * M / (vc**2) # Schwarzschild radius in m
r0  = 1.8e13  # Periastron distance in m. Commonly 1.80e13 is provided in literature, but this value, together with the velocity of 7.65e6 m/s,
x0 = np.sqrt(r0**2 + spin**2)
vphi0 = 7.65e6 / r0  # Periastron velocity in m/s
vtheta0 = 0 # orbit in equatorial plane
vt0_rel = 1/np.sqrt(1 - rs/r0)   # 1
tau_max = 150 * year  # simulate for 150 years (9 S2 orbits)
analyzePeriastron = True # should be 713 arsec per orbit
simulate_orbit()

__Prograde particle orbit around rotating black hole (yukterez.net)__
This constellation is to test the orbit solver with a known result. The results match to a large degree. The constellation does not require the 2-body approach (1-body is sufficient) as the mass ratio is 1e5.
https://upload.wikimedia.org/wikipedia/commons/6/6b/Orbit_um_ein_rotierendes_schwarzes_Loch_%28Animation%29.gif

In [ ]:
initialize_globals()
name = "Prograde particle orbit around rotating black hole (yukterez.net)"
# E=0.9352, Lz=2.3718, Q=3.8251
adjustZAMO = True
spin = 0.90 # Kerr metric's a parameter
r0 = 7.0  # Kerr metric (Wikipedia)
x0 = np.sqrt(r0**2 + spin**2)
vtheta0 = 0.256074/r0 # local velocity of 0.4, i0 = arctan(5/6)
vphi0 = 0.307289/x0 # note x0 here (but not for vtheta)
tau_max = 953 * year  # simulation period
analyzePeriastron = True # should be 713 arsec per orbit
simulate_orbit()

__Retrograde particle orbit around rotating black hole (yukterez.net)__
This constellation is to test the orbit solver with a known result. The results match to a large degree. The constellation does not require the 2-body approach (1-body is sufficient) as the mass ratio is 1e5.
https://upload.wikimedia.org/wikipedia/commons/7/74/Orbit_around_a_rotating_Kerr_black_hole.gif

In [ ]:
initialize_globals()
name = "Retrograde particle orbit around rotating black hole (yukterez.net)"
# E=0.9565, Lz=-0.8303, Q=13.413
adjustZAMO = True
spin = 0.95 * M # Kerr metric's a parameter
r0 = 6.5  # Kerr metric (Wikipedia)
x0 = np.sqrt(r0**2 + spin**2)
vtheta0 = -np.cos(11/50)/2/r0 # local velocity of 0.5
vphi0 = -np.sin(11/50)/2/x0 # note x0 here (but not for vtheta)
tau_max = 1640 * year  # simulation period
analyzePeriastron = True # should be 713 arsec per orbit
simulate_orbit()

__Photon bending around a black hole, getting back to its origin__  
This is a hypothetical constellation, resulting in a highly deflected light ray

In [ ]:
initialize_globals()
name = "Photon bending around a black hole, getting back to its origin"
ds0 = 0.0 # null geodesic
spin = 0.0285 # Parameter chosen such that photon gets back to origin
r0 = 53  # Starting point near the black hole
x0 = np.sqrt(r0**2 + spin**2)
vtheta0 = 0 #
vphi0 = 0.09/x0 # atan = 0.1
vr0 = -0.9 #atan = 0.1
tau_max = 120 * year  # simulation period
simulate_orbit()

__Photon wrapping around a black hole in a full orbit__  
This is a hypothetical constellation, resulting in a highly deflected light ray



In [ ]:
initialize_globals()
name = "Photon wrapping around a black hole in a full orbit"
ds0 = 0.0 # null geodesic
spin = 0 # Parameter chosen such that photon gets back to origin
r0 = 52.25  # Starting point near the black hole
x0 = np.sqrt(r0**2 + spin**2)
vtheta0 = 0 #
vphi0 = 0.09/x0 # atan = 0.1
vr0 = -0.9 #atan = 0.1
tau_max = 120 * year  # simulation period
simulate_orbit()

__Photon deflected by spin (1/2)__  
This is a hypothetical constellation, resulting in a highly deflected light ray

In [ ]:
initialize_globals()
name = "Photon deflected by spin (1/2)"
ds0 = 0.0 # null geodesic
spin = 1.0 # Parameter chosen such that photon gets back to origin
r0 = 53  # Starting point near the black hole
x0 = np.sqrt(r0**2 + spin**2)
vphi0 = 0 #
vtheta0 = -0.09/r0 # atan = 0.1
vr0 = -0.9 #atan = 0.1
tau_max = 120 * year  # simulation period
simulate_orbit()

__Photon deflected by spin (2/2)__  
This is a hypothetical constellation, resulting in a highly deflected light ray

In [ ]:
initialize_globals()
name = "Photon deflected by spin (2/2)"
ds0 = 0.0 # null geodesic
spin = 1.0 # Parameter chosen such that photon gets back to origin
r0 = 49.  # Starting point near the black hole
x0 = np.sqrt(r0**2 + spin**2)
vphi0 = 0 #
vtheta0 = -0.09/r0 # atan = 0.1
vr0 = -0.9 #atan = 0.1
tau_max = 120 * year  # simulation period
simulate_orbit()